In [1]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"
driver = GraphDatabase.driver(uri, auth=(username, password))

In [2]:
try:
    with driver.session() as session:
        result = session.run("RETURN 1 AS test")
    print("Connection successful")
except Exception as e:
    print(f"Connection failed: {e}")

Connection successful


In [3]:
import os
with open('./pathCSV.txt','r') as f:
    path = f.read()
os.path.exists(path)

True

In [4]:
business_path = "business.csv"
review_path = "review.csv"
# tip_path = "yelp_academic_dataset_tip.json"
user_path = "users.csv"

business_path, review_path, user_path = list(map(lambda y: path + '/' + y,[business_path, review_path, user_path]))
# business_path, review_path, user_path = list(map(lambda y:'"'+ path + '/' + y+'"',[business_path, review_path, user_path]))
print(business_path)

../filteredCSV/business.csv


In [10]:
import pandas as pd
import neo4j

users=pd.read_csv("usersAllFriends.csv")
# print(users)
users['friends']=users['friends'].fillna('').str.split(',')
print(type(users['friends'][0]))

users['friends']=users['friends'].apply(set)
print(type(users['friends'][0]))
# users=pd.read_csv(user_path)
# users['friends'].str.split(',')

n=5 
selectRandom="""
        MATCH (a)-[:REVIEWS]->() 
        RETURN a.user_id, rand() as r
        ORDER BY r Limit $n
    """
userList=driver.execute_query(selectRandom,n=n,database_="neo4j",result_transformer_=neo4j.Result.to_df)['a.user_id']
# print(userList)

for user in userList:
	trueSet=users[users['user_id']==user]['friends'].iloc[0]
	query = """
        MATCH (n1:User {user_id:$user})-[r1:REVIEWS]->(b:Business)<-[r2:REVIEWS]-(n2:User)

        WHERE -2 < r1.stars - r2.stars < 2 AND NOT (n1)-[:HAS_FRIEND]->(n2)
        WITH n1, n2, b
        match  (b)--()
        with n1,n2, count(*) AS bDegree
        WITH n1, n2, SUM(1.0 / log(bDegree)) AS adamicScore

                MATCH (n1)-[:REVIEWS]->(m:Business)-[:IN_CATEGORY]->(c:Category)<-[:IN_CATEGORY]-(m2:Business)<-[:REVIEWS]-(n2)
        WITH n1, n2, c, adamicScore, COUNT(*) AS sharedReviewCount
        match (c)--()
        WITH n1, n2, c, adamicScore, sharedReviewCount,count(*) AS cDegree
        WITH n1, n2, adamicScore, SUM(1.0 * sharedReviewCount / cDegree) AS categoryBonus
        where (adamicScore + categoryBonus)>0.6
        
        RETURN adamicScore + categoryBonus AS totalScore, n2.user_id
                """
	query1="""
	 MATCH (n1:User {user_id:$user})-[r1:HAS_FRIEND]->(n2:User)
	 return n1.user_id, n2.user_id 
"""
	result=driver.execute_query(query,user=user,database_="neo4j",result_transformer_=neo4j.Result.to_df)['n2.user_id']
	total=len(result)
	# print(total)
	correct=0
	# print("set",trueSet)
	# for element in trueSet:
	# 	print(element,type(element))
	# 	break

	# print("pred",result)
	for prediction in result:
		# print(prediction,type(prediction))
		# break
		if prediction in trueSet:
			correct+=1
	if total!=0:
		accuracy=100*correct/total
	else:
		accuracy= "NA"	
	print(f"total: {total}, correct: {correct}, accuract: {accuracy:.2f}%")
	# print(result)
    # alculate accuracy: how many of our predicted friends are actually friends ?


<class 'list'>
<class 'set'>
total: 26, correct: 15, accuract: 57.69%
total: 45, correct: 16, accuract: 35.56%
total: 44, correct: 27, accuract: 61.36%
total: 3, correct: 0, accuract: 0.00%
total: 295, correct: 131, accuract: 44.41%
